In [1]:
import sys,os
if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))
from dashgrid.dgrid_components import PlotlyCandles as plc

import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output,State
from dash.exceptions import PreventUpdate
import dash_table
import pandas as pd
import numpy as np
import json
import logging
import datetime
import functools
import random
import inspect



In [2]:
DEFAULT_LOG_PATH = './logfile.log'
DEFAULT_LOG_LEVEL = 'INFO'

def init_root_logger(logfile=DEFAULT_LOG_PATH,logging_level=DEFAULT_LOG_LEVEL):
    level = logging_level
    if level is None:
        level = logging.DEBUG
    # get root level logger
    logger = logging.getLogger()
    if len(logger.handlers)>0:
        return logger
    logger.setLevel(logging.getLevelName(level))

    fh = logging.FileHandler(logfile)
    fh.setLevel(logging.DEBUG)
    # create console handler with a higher log level
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    # create formatter and add it to the handlers
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    ch.setFormatter(formatter)
    # add the handlers to the logger
    logger.addHandler(fh)
    logger.addHandler(ch)   
    return logger


In [3]:
logger = init_root_logger(logging_level='DEBUG')


In [4]:
def stop_callback(errmess,logger=None):
    m = "****************************** " + errmess + " ***************************************"     
    if logger is not None:
        logger.debug(m)
    raise PreventUpdate()


# Dlink: Link Dash components 

In [5]:
def slice_df(df_in,input_list):
    """!
        ### slice_df slices a DataFrame according to a list of values.
        1. The arg ```input_list``` contains values that coincide with columns of the DataFrame
        2. The method returns a new filtered DataFrame where:
         * the method filters each column using the respective value in ```input_list```
         * if there is a ```None``` in any item in the input list, the method ignores filtering on that column
        :param df_in: DataFrame to be sliced
        :param input_list: list of values to use as criteria for each column slice. 
                            A value of None will cause the method to not slice that column
        :return: new DataFrame that has been sliced
    """
    dfc = df_in.drop_duplicates().copy()
    cols = dfc.columns.values
    current_input = 0
    for i in range(len(input_list)):
        il = input_list[i]
        col = cols[i]
        if il is not None:
            try:                
                dfc = dfc[dfc[col]==il]
            except Exception as e:
                raise ValueError(e)
        current_input += 1
    return dfc


In [52]:
def choices_from_df(df_in,input_list,col_to_select):
    '''
    Create a list of dicts that conform to an options list for a Dash componet like RadioItems or Dropdown,where:
    1. Each dict in the list has a 'label' key, and a 'value' key
    2. ```df_in``` is sliced
    '''
    dfc = slice_df(df_in,input_list)
    if dfc is None or len(dfc)<1:
        return None,None
    unique_values = dfc[col_to_select].unique()
    choices = unique_values[0],[{'label':uv,'value':uv} for uv in unique_values]
    return choices



In [7]:
GRID_STYLE = {'display': 'grid',
              'border': '1px solid #000',
              'grid-gap': '8px 8px',
              'background-color':'#fffff9',
            'grid-template-columns': '1fr 1fr'}

def create_grid(
        component_array,num_columns=2,
        column_width_percents=None,
        additional_grid_properties_dict=None,
        wrap_in_loading_state=False,
        row_layout=None):
    gs = GRID_STYLE.copy()
    if row_layout is None:
        percents = [str(round(100/num_columns-.001,1))+'%' for _ in range(num_columns)] if column_width_percents is None else [str(c)+'%' for c in column_width_percents]
        perc_string = " ".join(percents)
        gs['grid-template-columns'] = perc_string 
    else:
        gs['grid-template-columns'] = row_layout
    if additional_grid_properties_dict is not None:
        for k in additional_grid_properties_dict.keys():
            gs[k] = additional_grid_properties_dict[k]           
    
    div_children = []
    for c in component_array:
        if type(c)==str:
            div_children.append(GridItem(c).html)
        elif hasattr(c,'html'):
            div_children.append(c.html)
        else:
            div_children.append(c)
    if wrap_in_loading_state:
        g = dcc.Loading(html.Div(div_children,style=gs),type='cube')
    else:
        g = html.Div(div_children,style=gs)
    return g

class GridItem():
    def __init__(self,child,html_id=None):
        self.child = child
        self.html_id = html_id
    @property
    def html(self):
        if self.html_id is not None:
            return html.Div(children=self.child,className='grid-item',id=self.html_id)
        else:
            return html.Div(children=self.child,className='grid-item')


In [8]:
#********** define useful css styles ***********************
borderline = 'none' #'solid'
button_style={
    'line-height': '40px',
    'borderWidth': '1px',
    'borderStyle': borderline,
    'borderRadius': '1px',
    'textAlign': 'center',
    'background-color':'#fffff0',
    'vertical-align':'middle',
}
button_style_no_border={
    'line-height': '40px',
    'textAlign': 'center',
    'background-color':'#fffff0',
    'vertical-align':'middle',
}

blue_button_style={
    'line-height': '40px',
    'textAlign': 'center',
    'background-color':'#A9D0F5',#ed4e4e
    'vertical-align':'middle',
}

border_style={
    'line-height': '40px',
    'border':borderline + ' #000',
    'textAlign': 'center',
    'vertical-align':'middle',
}

table_like = {
    'display':'table',
    'width': '100%'
}

# define h4_like because h4 does not play well with dash_table
h4_like = { 
    'display': 'table-cell',
    'textAlign' : 'center',
    'vertical-align' : 'middle',
    'font-size' : '16px',
    'font-weight': 'bold',
    'width': '100%',
    'color':'#22aaff'
}

DEFAULT_TIMEZONE = 'US/Eastern'


In [9]:
def create_dt_div(dtable_id,df_in=None,
                  columns_to_display=None,
                  editable_columns_in=None,
                  title='Dash Table',logger=None,
                  title_style=None):
    '''
    Create an instance of dash_table.DataTable, wrapped in an dash_html_components.Div
    
    :param dtable_id: The id for your DataTable
    :param df_in:     The pandas DataFrame that is the source of your DataTable (Default = None)
                        If None, then the DashTable will be created without any data, and await for its
                        data from a dash_html_components or dash_core_components instance.
    :param columns_to_display:    A list of column names which are in df_in.  (Default = None)
                                    If None, then the DashTable will display all columns in the DataFrame that
                                    it receives via df_in or via a callback.  However, the column
                                    order that is displayed can only be guaranteed using this parameter.
    :param editable_columns_in:    A list of column names that contain "modifiable" cells. ( Default = None)
    :param title:    The title of the DataFrame.  (Default = Dash Table)
    :param logger:
    :param title_style: The css style of the title. Default is dgrid_components.h4_like.
    '''
    # create logger 
    lg = init_root_logger() if logger is None else logger
    
    lg.debug(f'{dtable_id} entering create_dt_div')
    
    # create list that 
    editable_columns = [] if editable_columns_in is None else editable_columns_in
    datatable_id = dtable_id
    dt = dash_table.DataTable(
        page_current= 0,
        page_size= 100,
        filter_action='none', # 'fe',
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ],
        style_cell_conditional=[
            {
                'if': {'column_id': c},
                'textAlign': 'left',
            } for c in ['symbol', 'underlying']
        ],

        style_as_list_view=False,
        style_table={
            'maxHeight':'450px','overflowX': 'scroll','overflowY':'scroll'
#             'height':'15','overflowX': 'scroll','overflowY':'scroll'
        } ,
        editable=True,
        css=[{"selector": "table", "rule": "width: 100%;"}],
        id=datatable_id
    )
    if df_in is None:
        df = pd.DataFrame({'no_data':[]})
    else:
        df = df_in.copy()
        if columns_to_display is not None:
            if any([c not in df.columns.values for c in columns_to_display]):
                m = f'{columns_to_display} are missing from input data. Your input Csv'
                raise BadColumnsException(m)           
            df = df[columns_to_display]
            
    dt.data=df.to_dict('rows')
    dt.columns=[{"name": i, "id": i,'editable': True if i in editable_columns else False} for i in df.columns.values]                    
    s = h4_like if title_style is None else title_style
    child_div = html.Div([html.Div(html.Div(title,style=s),style=table_like),dt])
    lg.debug(f'{dtable_id} exiting create_dt_div')
    return child_div


In [10]:
def create_xy_graph():
    pass

In [11]:
def recursive_grid_layout(app_component_list,current_component_index,gtcl,layout_components,wrap_in_loading_state=False):
    # loop through the gtcl, and assign components to grids
    for grid_template_columns in gtcl:
        if type(grid_template_columns)==list:
            layout_components.append(recursive_grid_layout(
                app_component_list,current_component_index, grid_template_columns, 
#                 layout_components,wrap_in_loading_state=True))
                layout_components,wrap_in_loading_state=False))
            continue
        # if this grid_template_columns item is NOT a list, process it normally
        sub_list_grid_components = []
        num_of_components_in_sublist = len(grid_template_columns.split(' '))
        for _ in range(num_of_components_in_sublist):
            # get the current component
            layout_ac = app_component_list[current_component_index]
            # add either the component, or it's html property to the sublist
            if hasattr(layout_ac, 'html'):
                layout_ac = layout_ac.html
            sub_list_grid_components.append(layout_ac)
            current_component_index +=1
        new_grid = create_grid(sub_list_grid_components, 
                        additional_grid_properties_dict={'grid-template-columns':grid_template_columns},
                        wrap_in_loading_state=wrap_in_loading_state)
        layout_components.append(new_grid)

def make_layout(comp_list,grid_template_columns_list):
    # for horizontal default
#     default_gtcl = [' '.join(['1fr' for _ in range(len(comp_list))])]
    # for vertical default
    default_gtcl = ['1fr' for _ in range(len(comp_list))]
    gtcl = default_gtcl if grid_template_columns_list is None else grid_template_columns_list
    
    # populate layout_components using recursive algo
    layout_list = []    
    recursive_grid_layout(comp_list,0,gtcl,layout_list)
    return    layout_list

In [101]:
class_converters = {
    dcc.Checklist:lambda v:v,
    dcc.DatePickerRange:lambda v:v,
    dcc.DatePickerSingle:lambda v:v,
    dcc.Dropdown:lambda v:v,
    dcc.Input:lambda v:v,
    dcc.Markdown:lambda v:v,
    dcc.RadioItems:lambda v:v,
    dcc.RangeSlider:lambda v:v,
    dcc.Slider:lambda v:v,
    dcc.Store:lambda v:v,
    dcc.Textarea:lambda v:v,
    dcc.Upload:lambda v:v,
}

html_members = [t[1] for t in inspect.getmembers(html)]
dcc_members = [t[1] for t in inspect.getmembers(dcc)]
all_members = html_members + dcc_members

class DashLink():
    def __init__(self,in_tuple_list, out_tuple_list,io_callback):
        _in_tl = [(k.id if type(k) in all_members else k,v) for k,v in in_tuple_list]
        _out_tl = [(k.id if type(k) in all_members else k,v) for k,v in out_tuple_list]
#         self.in_types = [type(t[0]) for t in in_tuple_list]        
#         self.input_converters = [class_converters[t] if t in  class_converters.keys() else (lambda v:v) for t in self.in_types]
        self.inputs = [Input(k,v) for k,v in _in_tl]
        self.outputs = [Output(k,v) for k,v in _out_tl]
        self.io_callback = io_callback
                       
    def callback(self,theapp):
        @theapp.callback(
            self.outputs,
            self.inputs
            )
        def execute_callback(*inputs_and_states):
            l = list(inputs_and_states)
            ret = self.io_callback(l)
            return ret if type(ret) is list else [ret]
#             lc = []
#             for i in range(len(self.in_types)):
#                 v = l[i]
#                 converter = self.input_converters[i]
#                 new_v = converter(v)
#                 lc.append(new_v)
#             ret = self.io_callback(lc)
#             return ret if type(ret) is list else [ret]
        return execute_callback
        
def makeapp(comp_list,dlink_list,grid_layout_list=None):
    app = dash.Dash()
#     grid_html = create_grid(comp_list)
    layout_components = make_layout(comp_list,grid_layout_list)
    grid_html = html.Div(layout_components,style={'margin-left':'10px','margin-right':'10px'})
    app.layout=html.Div(grid_html)
    for dlink in dlink_list:
        dlink.callback(app)
    return app

In [25]:
def generate_dataframe(column_names,possible_values_per_name,weights_per_name,num_rows):
    """
    :param column_schema: a list of dict objects where each element is:
        key/value: 'column_name':name
        key/value: 'possible_values':list of possible values
        key/value: 'weights_per_value': list of weights
        
    """
    col_data_lists = [] # this is a 2-d list
    for i in range(len(column_names)):
        possible_values = possible_values_per_name[i]
        weights = weights_per_name[i]
        choices = random.choices(possible_values,weights=weights,k=num_rows)
        col_data_lists.append(choices)
    # assemble dataframe
    data = {column_names[i]:col_data_lists[i] for i in range(len(column_names))}
    return pd.DataFrame(data)

In [26]:
def generate_random_walk_df(
        num_days_to_create=1000,
        end_date = None,
        annual_std_for_close=.15, 
        annual_std_for_volume=.5,
        initial_volume=1000000):
    all_days=num_days_to_create
    end_date = datetime.datetime.now() if end_date is None else end_date
    beg_date = end_date - datetime.timedelta(all_days)
    date_series = pd.bdate_range(beg_date,end_date)
    trade_dates = date_series.astype(str).str.replace('-','').astype(int)
    n = len(trade_dates)
    changes = np.random.lognormal(0,annual_std_for_close/256**.5,n-1)
    initial = np.array([100.0])
    closes = np.cumprod(np.append(initial,changes)).round(2)
    open_ranges = np.random.lognormal(0,.3/256**.5,n)
    opens = (closes * open_ranges).round(2)
    low_ranges = np.random.lognormal(.1,.2/256**.5,n)
    lows = np.array([min(x,y) for x,y in zip(opens,closes)]) - low_ranges
    lows = lows.round(2)
    high_ranges = np.random.lognormal(.1,.2/256**.5,n)
    highs = np.array([max(x,y) for x,y in zip(opens,closes)]) + high_ranges
    highs = highs.round(2)

    volume_changes = np.random.lognormal(0,annual_std_for_volume/256**.5,n-1)
    initial_volumes = np.array([initial_volume]) * np.random.lognormal(0,annual_std_for_volume/256**.5)
    volumes = np.cumprod(np.append(initial_volumes,volume_changes)).round(0)


    df_pseudo = pd.DataFrame({'date':trade_dates,'open':opens,'high':highs,'low':lows,
                      'close':closes,'volume':volumes})
    return df_pseudo



In [49]:
def generate_random_breakfast_times():
    
    rows = 40
    col_names = ['name','day','breakfast_time','minute']

    names = ["Sarah", "Billy", "Michael"]
    days = ["Monday", "Tuesday","Wednesday",'Thursday','Friday']
    times = list(range(7,12))
    minutes = list(range(9,17))
    possible_values_list = [names,days,times,minutes]

    name_weights = [2, 1, 1]
    day_weights = [2,1,2,1,1]
    time_weights = np.ones(len(times))
    minute_weights = np.ones(len(minutes))
    weights_list = [name_weights,day_weights,time_weights,minute_weights]

    df2 = generate_dataframe(col_names,possible_values_list,weights_list,rows)
    return df2

### Example Apps

In [50]:
def pick_from_random_lists_app():
    df2 = generate_random_breakfast_times()

    rr1_ops = choices_from_df(df2,[None]*3,df2.columns.values[0])
    rr1 = dcc.RadioItems('rr1',options=rr1_ops[1],value=rr1_ops[0])

    dd1 = dcc.Dropdown('dd1',options=[])
    dd1_link = DashLink([(rr1.id,'value')],[(dd1.id,'value'),(dd1.id,'options')],
                      lambda input_list:list(choices_from_df(df2,input_list,df2.columns.values[1])))

    rr2 = dcc.RadioItems('rr2')
    rr2_link = DashLink([(rr1.id,'value'),(dd1.id,'value')],[(rr2.id,'value'),(rr2.id,'options')],
                      lambda input_list:list(choices_from_df(df2,input_list,df2.columns.values[2])))

    dd2 = dcc.Dropdown('dd2',options=[])
    dd2_link = DashLink([(rr1.id,'value'),(dd1.id,'value'),(rr2.id,'value')],[(dd2.id,'value'),(dd2.id,'options')],
                      lambda input_list:list(choices_from_df(df2,input_list,df2.columns.values[3])))

    dt1 = create_dt_div('dt1',df_in=df2,title='My Table')
    dt1_link = DashLink([(rr1.id,'value'),(dd1.id,'value'),(rr2.id,'value')],[('dt1','data')],
                      lambda input_list:[slice_df(df2,input_list).to_dict('records')])
    
    gtcl = ['1fr 1fr 1fr 1fr','1fr']
    app = makeapp([rr1,dd1,rr2,dd2,dt1],[dd1_link,rr2_link,dd2_link,dt1_link],gtcl)
    app.run_server(host='127.0.0.1',port=8500)
   

In [45]:
def value_componets_example():
    days = ["Monday", "Tuesday","Wednesday",'Thursday','Friday']
    ch1 = dcc.Checklist(id='ch1',options=[{'label':v,'value':v} for v in days],value=[days][2:4])
    sl1 = dcc.RangeSlider(id='sl1',min=-5,max=10,step=0.5,value=[-3,5])
    
    div1 = html.Div(children=[],id='div1')    
    div1_link = DashLink([(ch1,'value'),(sl1,'value')],[(div1,'children')],
                         lambda input_list:' , '.join([str(v) for v in input_list]))

    gtcl = ['1fr 1fr 1fr']
    app = makeapp([ch1,sl1,div1],[div1_link],gtcl)
    app.run_server(host='127.0.0.1',port=8500)


In [114]:
def random_market_data_app():
    dfr = generate_random_walk_df()
    lb1 = html.Label("Enter Begin yyyymmdd (e.g. 20180524)")
    in1 = dcc.Input(id='in1',placeholder='Enter a begining yyyymmdd',type='number',value=dfr.date.values[-50],debounce=True)
    lb2 = html.Label("Enter Ending yyyymmdd (e.g 20181124)")
    in2 = dcc.Input(id='in2',placeholder='Enter a ending yyyymmdd',type='number',value=dfr.date.values[-1],debounce=True)
    dt1 = create_dt_div('dt1',df_in=dfr,title='My Table')
    
    dt1_link = DashLink(
        [(in1,'value'),(in2,'value')],
        [('dt1','data')],
#         lambda input_list:[dfr[(dfr.date>=input_list[0][0]) & (dfr.date<=input_list[1][0])].to_dict('records')])
        lambda input_list:[dfr[(dfr.date>=input_list[0]) & (dfr.date<=input_list[1])].to_dict('records')])

    def _makegraph(input_list):
        dict_df = input_list[0]
        df = pd.DataFrame(dict_df)
        fig1 = plc(df,number_of_ticks_display=20,title='Random Walk').get_figure()
        gr1 = dcc.Graph(id='gr1',figure=fig1)
        return gr1
        
    div1 = html.Div(children=[],id='div1')
    div1_link = DashLink([('dt1','data')],[('div1','children')],_makegraph)
                                         
    gtcl = ['1fr 1fr 1fr 1fr','1fr', '1fr']
    app = makeapp([lb1,in1,lb2,in2,div1,dt1],[dt1_link,div1_link],gtcl)
    app.run_server(host='127.0.0.1',port=8500)


In [113]:
def dropdown_to_df_app():
    dfr = generate_random_breakfast_times()
    dd_names = dcc.Dropdown(id='dd_names',options=choices_from_df(dfr,[None]*3,dfr.columns.values[0])[1])
    dd_days = dcc.Dropdown(id='dd_days',options=choices_from_df(dfr,[None]*3,dfr.columns.values[1])[1])
    dd_times = dcc.Dropdown(id='dd_times',options=choices_from_df(dfr,[None]*3,dfr.columns.values[2])[1])
    
    dt1 = create_dt_div('dt1',df_in=dfr,title="Breakfast times")
    def _slice_dfr(input_list):
        dfr2 = dfr.copy()
        cols =  dfr2.columns.values
        for i in range(len(input_list)):
            dfr2 = dfr2 if input_list[i] is None else dfr2[dfr2[cols[i]]==input_list[i]]
        return [dfr2.to_dict('records')]

    dt1_link = DashLink([(dd_names,'value'),(dd_days,'value'),(dd_times,'value')],[('dt1','data')],_slice_dfr)


    def _get_choices(input_list,col_num):
        dict_df = input_list[0]
        if dict_df is None:
            return None
        dft = pd.DataFrame(dict_df)
        col = dft.columns.values[col_num]
        value_and_choices = choices_from_df(dft,[None,None,None],col)
        ret =  [value_and_choices[1]]
        return ret

    dd_names_link = DashLink([('dt1','data')],[(dd_names,'options')],lambda input_list:_get_choices(input_list,0))
    dd_days_link = DashLink([('dt1','data')],[(dd_days,'options')],lambda input_list:_get_choices(input_list,1))
    dd_times_link = DashLink([('dt1','data')],[(dd_times,'options')],lambda input_list:_get_choices(input_list,2))

    gtcl = ['1fr 1fr 1fr','1fr']
    app = makeapp([dd_names,dd_days,dd_times,dt1],[dt1_link,dd_names_link,dd_days_link,dd_times_link],gtcl)
    app.run_server(host='127.0.0.1',port=8500)
    

In [110]:
choices_from_df(dfr,[None,None,None],'name')[1]

[{'label': 'Billy', 'value': 'Billy'},
 {'label': 'Sarah', 'value': 'Sarah'},
 {'label': 'Michael', 'value': 'Michael'}]

### Run examples

In [118]:

if __name__=='__main__':
    app_list = [
        pick_from_random_lists_app,
        random_market_data_app,
        value_componets_example,
        dropdown_to_df_app
    ]
    app_list[3]()


2020-02-09 21:57:38,717 - root - DEBUG - dt1 entering create_dt_div
2020-02-09 21:57:38,720 - root - DEBUG - dt1 exiting create_dt_div


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2020-02-09 21:57:38,732 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
2020-02-09 21:57:40,647 - werkzeug - INFO - 127.0.0.1 - - [09/Feb/2020 21:57:40] "GET / HTTP/1.1" 200 -
2020-02-09 21:57:40,725 - werkzeug - INFO - 127.0.0.1 - - [09/Feb/2020 21:57:40] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-02-09 21:57:40,726 - werkzeug - INFO - 127.0.0.1 - - [09/Feb/2020 21:57:40] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-02-09 21:57:40,737 - werkzeug - INFO - 127.0.0.1 - - [09/Feb/2020 21:57:40] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-02-09 21:57:40,740 - werkzeug - INFO - 127.0.0.1 - - [09/Feb/2020 21:57:40] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.0.0&m=1574289294 HTTP/1.1" 200 -
2020-02-09 21:57:40,761 - werkzeug - INFO - 127.0.0.1 

2020-02-09 21:59:53,508 - werkzeug - INFO - 127.0.0.1 - - [09/Feb/2020 21:59:53] "POST /_dash-update-component HTTP/1.1" 200 -


In [20]:

['Checklist',
 'DatePickerRange',
 'DatePickerSingle',
 'Dropdown',
 'Graph',
 'Input',
 'Markdown',
 'RadioItems',
 'RangeSlider',
 'Slider',
 'Store',
 'Textarea',
 'Upload']



['Checklist',
 'DatePickerRange',
 'DatePickerSingle',
 'Dropdown',
 'Graph',
 'Input',
 'Markdown',
 'RadioItems',
 'RangeSlider',
 'Slider',
 'Store',
 'Textarea',
 'Upload']

In [21]:
# !jupyter nbconvert --to script easycomp.ipynb

# End